# Optimal Portoflio

This notebook demonstrates how to use [Markowitz Portfolio](https://www.math.hkust.edu.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf) optimization to setup an optimal portfolio using the MultiInstrumentEnv.

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import deeptrade.env
from deeptrade.env import HoldAgent, EWMACAgent, BreakoutAgent
import deeptrade.util.finance as futils

COLOURS = [[0, 18, 25], [0, 95, 115], [10, 147, 150], [148, 210, 189], [233, 216, 166], [238, 155, 0], [202, 103, 2], [187, 62, 3], [174, 32, 18], [155, 34, 38]]
COLOURS = [[value/255 for value in rgb] for rgb in COLOURS]

In [29]:
seed = 0
n_instruments = 5
starting_prices = [10000.0, 10000.0, 10000.0, 10000.0, 10000.0]
drifts = [0.0011, 0.0012, 0.009, 0.0011, 0.0012]
vars = [0.08, 0.04, 0.05, 0.09, 0.01]
n_days = 365
price_gen_info = {
    "starting_prices": starting_prices,
    "means": drifts,
    "vars": vars,
    "n_days": n_days, 
}
env = gym.make("MultiInstrument-v0", seed=seed, n_instruments = 5, price_gen_info=price_gen_info)

In [ ]:
price_data = env.unwrapped.prices_data

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
for i in range(n_instruments):
    ax.plot(price_data[i], label=f"Instrument {i}", color=COLOURS[i+1])
ax.grid()
ax.legend()
ax.set_ylabel("Price [USD]")
ax.set_xlabel("Time [days]")
fig.show()

In [ ]:
from pypfopt import risk_models
from pypfopt import plotting

dates = pd.date_range(start='1990-01-01', periods=price_data.shape[1], freq='D')
prices = pd.DataFrame(price_data.T, index=dates, columns=[f'Asset {i+1}' for i in range(price_data.shape[0])])
sample_cov = risk_models.sample_cov(prices, frequency=252)
sample_cov

In [ ]:
plotting.plot_covariance(sample_cov, plot_correlation=True)

In [ ]:
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
plotting.plot_covariance(S, plot_correlation=True)

In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
mu = expected_returns.capm_return(prices)
ef = EfficientFrontier(mu, S)
print(mu)

n_samples = 10000
w = np.random.dirichlet(np.ones(len(mu)), n_samples)
rets = w.dot(mu)
stds = np.sqrt((w.T * (S @ w.T)).sum(axis=0))
sharpes = rets / stds


fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False, verbose=True)

ef2 = EfficientFrontier(mu, S)
ef2.max_sharpe()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
print(ret_tangent, std_tangent)
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")
ax.set_title("Efficient Frontier with random portfolios")
ax.legend()
ax.grid()
plt.tight_layout()
plt.show()


In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
dates = pd.date_range(start='1990-01-01', periods=price_data.shape[1], freq='D')
prices = pd.DataFrame(price_data.T, index=dates, columns=[f'Asset {i+1}' for i in range(price_data.shape[0])])
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


latest_prices = get_latest_prices(prices)

da = DiscreteAllocation(weights=cleaned_weights, latest_prices=latest_prices, total_portfolio_value=1000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))


In [ ]:

prices = env.unwrapped.prices_data
simple_returns = []
for idi in range(n_instruments):
    simple_returns.append(futils.calculate_simple_returns(prices[idi]))

def pretty_print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{elem:>5.7f}" for elem in row))

simple_returns = np.array(simple_returns)
covm = np.cov(simple_returns)
rets = np.mean(simple_returns, axis=1)
pretty_print_matrix(covm)
print(rets)
plt.plot(futils.calculate_log_returns(prices[0, :]))

from scipy.optimize import minimize

def get_ef_numerically(rets, covm, targ: float = 0.00):
    
    def objective(weights):
        return weights.T @ covm @ weights - targ * rets.T @ weights
    
    norm_constraint = lambda weights: 1 - weights.sum()
    targ_constraint = lambda weights: np.dot(rets, weights) - targ
    
    resp = minimize(objective,
                    x0=np.random.dirichlet([1]*len(rets)),
                    method='SLSQP',
                    bounds=[(0, 2)],
                    constraints=[{'type': 'eq', 'fun': norm_constraint}]
                    )
    weights = resp.x
    
    return weights

w = get_ef_numerically(rets, covm, 0.05)
print(w, w.sum())
print(f"mu_p: {w.T @ rets}")
print(f"sigma_p: {w.T @ covm @ w}")

agents = {}
for instrument in range(n_instruments):
    size = w[instrument] * 10.0
    print(w[instrument], instrument)
    agents[instrument] = HoldAgent(env, pos_size=size, instrument=instrument)

In [ ]:
terminated, truncated = False, False
observations = []
actions = []
times = []
obs, _ = env.reset(seed=seed)
while (not terminated) and (not truncated):
    action = np.zeros(n_instruments)
    for instrument, agent in agents.items():
        action[instrument] = agent.act(obs)[instrument] * w[instrument]
    obs, reward, terminated, truncated, _ = env.step(action)
    observations.append(obs)
    actions.append(action)
    times.append(env.unwrapped.time)

observations = np.array(observations)
actions = np.array(actions)

In [ ]:
margins = [obs['margin'] for obs in observations]
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(times, margins, label='Margin')
# for idp in range(n_instruments):
    # ax.plot(times, prices[idp, :len(times)], label=f"Position {idp}")
# ax.plot(times, prices[0, :len(times)])